## Configuraciones Iniciales


In [4]:
import sys
import os
import pandas as pd

# Agregar la ruta del directorio scripts
sys.path.append(os.path.abspath(os.path.join('..', 'scripts')))

# Configuración de directorios
RAW_DIR = r'D:\Henry Data Science\Proyecto_individual_1\Datasets'
PROCESSED_DIR = r'D:\Henry Data Science\Proyecto_individual_1\Data'

# Crear directorio de destino si no existe
os.makedirs(PROCESSED_DIR, exist_ok=True)

### Conversión de archivos CSV a Parquet.
- La conversión del archivo CSV a Parquet optimiza el manejo de datos y reduce el tamaño del archivo.

In [5]:
from convert_csv_to_parquet import convertir_csv_a_parquet

# Convertir archivo CSV a formato Parquet
convertir_csv_a_parquet(
    os.path.join(RAW_DIR, 'credits.csv'),
    os.path.join(RAW_DIR, 'credits.parquet')
)

# Leer el archivo Parquet convertido
df = pd.read_parquet(os.path.join(RAW_DIR, 'credits.parquet'))
df.head(3)

Archivo convertido correctamente y guardado en: D:\Henry Data Science\Proyecto_individual_1\Datasets\credits.parquet


,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602


### Desanidado de la Columna cast

In [6]:
from convert_str_to_list_dic import convert_list_dict
from dislodge_columns import desanidar_columna

# Convertir columna 'cast' de string a lista de diccionarios
df['cast'] = df['cast'].apply(convert_list_dict)

# Desanidar la columna 'cast'
cast_desanidado = desanidar_columna(df, 'cast', 'cast_desanidado')

# Agregar columna 'id' del DataFrame original a 'cast_desanidado'
cast_desanidado = cast_desanidado.merge(df[['cast_id', 'id']], on='cast_id', how='left', suffixes=('', '_df'))

# Eliminar filas con valores nulos en columnas relevantes
columnas_con_nulos = ['character', 'credit_id', 'gender', 'id', 'name', 'order']
cast_desanidado = cast_desanidado.dropna(subset=columnas_con_nulos)

# Estandarizar datos en columnas clave
cast_desanidado['name'] = cast_desanidado['name'].str.lower()
cast_desanidado['id'] = cast_desanidado['id'].astype(int)

# Eliminar columnas innecesarias
cast_desanidado = cast_desanidado.drop(columns=['character', 'profile_path', 'order'])

### Desanidado de la Columna crew

In [7]:
# Convertir columna 'crew' de string a lista de diccionarios
df['crew'] = df['crew'].apply(convert_list_dict)

# Desanidar la columna 'crew'
crew_desanidado = desanidar_columna(df, 'crew', 'crew_desanidado', 1)

# Agregar columna 'id' del DataFrame original a 'crew_desanidado'
crew_desanidado = crew_desanidado.merge(df[['crew_id', 'id']], on='crew_id', how='left', suffixes=('', '_df'))

# Eliminar filas con valores nulos en columnas relevantes
columnas_con_nulos = ['credit_id', 'department', 'gender', 'id', 'job', 'name']
crew_desanidado = crew_desanidado.dropna(subset=columnas_con_nulos)

# Filtrar solo los datos de directores y estandarizar datos
crew_desanidado = crew_desanidado[crew_desanidado['job'] == 'Director']
crew_desanidado['name'] = crew_desanidado['name'].str.lower()
crew_desanidado['id'] = crew_desanidado['id'].astype(int)

# Eliminar columnas redundantes
crew_desanidado = crew_desanidado.drop(columns=['department', 'profile_path'])

### Preparación para la exportación.
- Convertir listas de nuevo a cadenas para garantizar compatibilidad al exportar.

In [8]:
from convert_list_to_str import lista_a_str

# Convertir listas de nuevo a strings para exportar
df['cast'] = df['cast'].apply(lista_a_str)
df['crew'] = df['crew'].apply(lista_a_str)

### Exportación de Tablas Procesadas

In [9]:
# Exportar cast desanidado
cast_path = os.path.join(PROCESSED_DIR, 'cast_desanidado.parquet')
cast_desanidado.to_parquet(cast_path, engine='pyarrow', compression='snappy', index=False)
print("Datos de 'cast_desanidado' exportados correctamente.")

# Exportar crew desanidado
crew_path = os.path.join(PROCESSED_DIR, 'crew_desanidado.parquet')
crew_desanidado.to_parquet(crew_path, engine='pyarrow', compression='snappy', index=False)
print("Datos de 'crew_desanidado' exportados correctamente.")

Datos de 'cast_desanidado' exportados correctamente.
Datos de 'crew_desanidado' exportados correctamente.
